In [ ]:
# 기사 목록

from selenium import webdriver
#from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 키워드와 검색 기간 설정
keywords = [
    "검색어",
]
start_date = "2024-06-01"
end_date = "2024-08-31"

seen_urls = set()

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def fetch_search_results(keyword, start_date, end_date):
    """지정된 검색어와 날짜 범위로 강원일보에서 기사 스크래핑."""
    base_url = "https://www.kwnews.co.kr/search?page="
    results = []
    page = 1  # 페이지 시작 번호 설정

    while True:
        print(f"{keyword} {page} 페이지 수집 중...")
        search_url = (
            f"{base_url}{page}&search={urllib.parse.quote(keyword)}"
            f"&start_date={start_date}&end_date={end_date}"
        )
        driver.get(search_url)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기사 목록 가져오기
        articles_section = soup.find("ul", id="search_list")
        if articles_section is None or not articles_section.find_all("li"):
            print(f"{page} 페이지에 더 이상 기사가 없습니다.")
            break

        # 기사 정보 추출
        for item in articles_section.find_all("li"):
            try:
                # 제목과 링크 추출
                title_tag = item.select_one("p.title a")
                if title_tag:
                    title = title_tag.text.strip()
                    url = "https://www.kwnews.co.kr" + title_tag["href"]
                else:
                    continue  # 제목이 없으면 다음 기사로 넘어감

                # 중복 URL 체크
                if url in seen_urls:
                    continue  # 중복된 URL 무시
                seen_urls.add(url)  # 새로운 URL 저장

                # 날짜 추출
                date = item.select_one('.date').text.strip().split()[0] if item.select_one('.date') else ""

                # 기사 결과 추가
                results.append({
                    'media': '강원일보',
                    'title': title,
                    'date': date,
                    'section': '',
                    'article_type': '',
                    'writer': '',
                    'page': '',
                    'url': url,
                    'content': ''  # 컨텐츠 정보 없음
                })
            except Exception as e:
                print(f"기사 정보 추출 중 오류 발생: {e}")

        page += 1  # 다음 페이지로 이동

    return results

# 엑셀 파일 저장 함수
def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

# 모든 결과 수집 및 중복 제거
all_results = []

try:
    for keyword in keywords:
        # 매개변수 전달
        results = fetch_search_results(keyword, start_date, end_date)
        all_results.extend(results)

    # DataFrame 생성 및 중복 제거
    df = pd.DataFrame(all_results)

    # 기사 수 출력
    print(f"Total number of articles: {len(all_results)}")

    # 엑셀 파일로 저장
    save_to_excel(all_results, f'2강원일보 {start_date}~{end_date}.xlsx')

finally:
    # 드라이버 종료
    driver.quit()



In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# content, section, page 열을 문자열 타입으로 미리 설정합니다.
df['content'] = ""
df['section'] = ""
df['page'] = ""
df['sub-title'] = ""

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 섹션, 페이지 정보를 스크랩하는 함수 정의
def get_content_section_and_page_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 섹션 정보 추출
        section = None
        article_head = soup.find('div', class_='article_head')
        if article_head:
            section_tag = article_head.find('p', class_='cate')
            if section_tag:
                section = section_tag.get_text(strip=True)
        
        # 페이지 정보 추출
        page = None
        date_tag = soup.find('span', class_='date')
        if date_tag:
            date_text = date_tag.get_text(strip=True)
            page_match = re.search(r'\((.+?)\)', date_text)
            if page_match:
                page = page_match.group(1)

        # 서브 타이틀 추출 (p.subtitle)
        subtitle = ""
        subtitle_tag = soup.find('p', class_='subtitle')
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True)
            subtitle_tag.decompose()  # 본문에서 서브 타이틀 제거        
        
        article_body = soup.find('div', class_="article_content")
        if article_body:
            # 'figure' 태그를 제거합니다.
            for figure in article_body.find_all('figure'):
                figure.decompose()
            util = article_body.find('div', class_='util')
            if util:
                next_sibling = util.find_next_sibling()
                while next_sibling:
                    next_sibling.decompose()
                    next_sibling = util.find_next_sibling()
                util.decompose()  # btn_area 자체도 제거    

            # article_view 클래스 내부의 전체 텍스트 추출
            content = article_body.get_text(separator='\n', strip=True)
            return content, section, page, subtitle

        return content, section, page, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None, None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문 내용을 스크랩하여 content, section, page 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, section, page, subtitle = get_content_section_and_page_from_url(row['url'])
    df.at[index, 'content'] = content if content is not None else ""
    if content.startswith('='):
        content = "'" + content        # content를 문자열로 변환하여 저장
    df.at[index, 'section'] = section if section is not None else ""
    df.at[index, 'page'] = page if page is not None else ""
    df.at[index, 'sub-title'] = subtitle if subtitle is not None else ""

            # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")
